<a href="https://colab.research.google.com/github/skyloma/img2/blob/master/TPU_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 加载google云盘

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# 切换目录


In [0]:
import os
os.chdir("drive/My Drive/models/research")


In [0]:
!ls

# 授权访问google cloud partform

In [8]:
from google.colab import auth
auth.authenticate_user()
project_id = 'lomasky8090'
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [0]:
!gcloud auth application-default login

# 复制文件到google storage

In [0]:
bucket_name="lomasky8090_data"
# !gsutil cp -r /content/drive/My\ Drive/ssd_mobilenet_v1_ppn_shared_box_predictor_300x300_coco14_sync_2018_07_03/ gs://{bucket_name}/data
!gsutil cp -r /content/drive/My\ Drive/models/research/data gs://{bucket_name}
# !gsutil cp test.record gs://{bucket_name}/data/test.record  
# !gsutil cp pk_rmb.pbtxt gs://{bucket_name}/data/pk_rmb.pbtxt    


# 切换到目录research下进行训练

In [0]:
from tensorboardcolab import TensorBoardColab
tbc=TensorBoardColab(graph_path='gs://lomasky8090_data/tpu128')

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://29da42d8.ngrok.io


In [0]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/model_tpu_main.py \
--gcp_project=lomasky8090  \
--pipeline_config_path=data/tpu128.config \
--model_dir=gs://lomasky8090_data/tpu128 \
--num_train_steps=50000 \
--num_eval_steps=8000

In [0]:

# !export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/model_tpu_main.py \
# --logtostderr \
# --pipeline_config_path=/content/drive/My Drive/tpu128.config \
# --model_dir=gs://mytf9133.appspot.com/train128 \
# --num_train_steps=200000 \
# --num_eval_steps=8000

In [0]:
# !export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/model_main.py \
# --logtostderr \
# --pipeline_config_path=data/gpu128.config \
# --model_dir=gpu128 \
# --num_train_steps=50000 \
# --num_eval_steps=8000

# 导出训练生成的文件

In [0]:
bucket_name="mytf27297_storage"
!gsutil cp -r  gs://{bucket_name}/tpu224  /content/drive/My\ Drive/outdata
  
# !gsutil cp  gs://{bucket_name}/data/pk_rmb.pbtxt A/
  
# !gsutil cp  gs://{bucket_name}/A/tflite_graph.pbtxt A/  

# 导出能转换成tflte的pb

In [0]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=data/tpu224.config \
--trained_checkpoint_prefix=/content/drive/My\ Drive/outdata/tpu224/model.ckpt-58600 \
--output_directory=/content/drive/My\ Drive/outmodel/tpu224 \
--add_postprocessing_op=true

# 转换没有量化的tflite

In [0]:
# !tflite_convert \
#   --graph_def_file=gs://mytf9133.appspot.com/train/output/tflite_graph.pb \
#   --input_format=TENSORFLOW_GRAPHDEF \
#   --output_format=TFLITE \
#   --output_file=gs://mytf9133.appspot.com/train/output/tflite_graph.tflite \
#   --inference_type=FLOAT \
#   --input_shapes=1,128,128,3 \
#   --input_arrays=normalized_input_image_tensor \
#   --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
#   --allow_custom_ops \

# 转换量化的tflite

In [0]:
!tflite_convert \
  --graph_def_file=/content/drive/My\ Drive/outmodel/tpu224/tflite_graph.pb \
  --output_format=TFLITE \
  --output_file=/content/drive/My\ Drive/outmodel/tpu224/tflite_graph52.tflite \
  --input_arrays=normalized_input_image_tensor \
  --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
  --input_shapes=1,224,224,3 \
  --inference_type=QUANTIZED_UINT8 \
  --mean_value=128 \
  --std_dev_values=128 \
  --default_ranges_min=0 \
  --default_ranges_max=6 \
  --change_concat_input_ranges=false \
  --allow_custom_ops
 

In [0]:
%cd /content 

# 下载模型到mytf4930工程 解压copy到gs://mytf_storage/data/

In [0]:
# cd /tmp
# !curl -O http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz
# !tar xzf ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz

!gsutil cp -r ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/  gs://lomasky8090_data/data/

In [0]:
import os
os.chdir("object_detection")


In [0]:
!git clone https://github.com/skyloma/img2.git  

In [0]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python xml_to_csv.py   --image_dir=img2/

In [0]:
%cd /content  
import os
os.chdir("drive/My Drive/models/research")


In [0]:
!ls

In [0]:
! export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim;python object_detection/generate_tfrecord52.py \
    --csv_input=object_detection/img2/train_labels.csv \
    --image_dir=object_detection/img2/train \
    --output_path=data/train.record  
!  export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/generate_tfrecord52.py \
    --csv_input=object_detection/img2/test_labels.csv \
    --image_dir=object_detection/img2/test \
    --output_path=data/test.record  

In [0]:
import os
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print(TPU_WORKER)
 

AttributeError: ignored